In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
import simpler_sf

simpler_sf.simple_salesforce()

In [2]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [4]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

100%|██████████| 1/1 [00:11<00:00, 11.30s/it]


,Id,Empresa__r.Id,Empresa__r.Name,Empresa__r.Holding__c,Empresa__r.Campa_a__c,Empresa__r.Id_Empresa__c,Empresa__r.Empleador__c,Empresa__r.poliza__c,Empresa__r.RUT_Empresa__c,Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c,Name
0,a1W8c000007c5aIEAQ,0018c00002k9KwBAAU,Aldo Conti MX,-,Aldo Conti,EMP10376,Aldo Conti MX,-,-,-,False,A-9930
1,a1W8c000007c669EAA,0018c00002k9yOCAAY,Caja de Compensación de Asignación Familiar de...,Itaú,Vida Security,EMP10377,Caja de Compensación de Asignación Familiar de...,283144,81826800-9,-,False,A-9931
2,a1W8c000007c66AEAQ,0018c00002k9yODAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10378,Itaú Corpbanca,283147,97023000-9,-,False,A-9932
3,a1W8c000007c66BEAQ,0018c00002k9yOEAAY,Itaú Corpbanca,Itaú - sin servicio,Vida Security,EMP10379,Itaú Corpbanca,283148,-,-,False,A-9933
4,a1W8c000007c66CEAQ,0018c00002k9yOFAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10380,Itaú Corpbanca,283149,-,-,False,A-9934


vllamadaEmpresa.shape


In [5]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(8232, 8)

In [6]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7912, 5)

In [7]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(8238, 10)

In [8]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [9]:
query2 = "select P_liza__c, Nombre_de_Empresa__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

100%|██████████| 1/1 [00:10<00:00, 10.63s/it]


,P_liza__c,Nombre_de_Empresa__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,True,a3yRN0000006mmjYAA,775173998Sin Póliza Asociada,1
1,-,Apunto Servicios De Alimentacion,-,Consalud,Holding Consalud,-,Chile,False,a3yRN0000006rG9YAI,-,1
2,-,2M Inmobiliaria Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGAYAY,-,1
3,-,360 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGBYAY,-,1
4,-,361 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGCYAY,-,1


fin empresas y comienzo de rescate de pacientes


In [10]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId":"Id Contacto"
    }
)



Pacientes_SF.shape

In [ ]:
def siPrimerCero(y):
    if len(y) <= 7:
        return "NONE"
    
    if y[0] == "0":
        y = y[1:]
        
    return y

In [ ]:

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

In [ ]:
Pacientes_Nones1 = Pacientes_SF[(Pacientes_SF["Rut"] == "NONE") ]
Pacientes_SF = Pacientes_SF[(Pacientes_SF["Rut"] != "NONE") ]
Pacientes_Nones1.to_excel("F:/Empresas SF/reportes/NONE1.xlsx", index=False) 

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

Carga de Archivo carga


In [181]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Construmart"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,Empresa,rut2,rut1,sexo,ParentesCONYUGE,nombres,apellidos,nacimiento,email,Relacion,Activo,Llave
0,Construmart,11990788-8,11990788-8,M,TITULAR,Ignacio Enrique,Arancibia Silva,1972-03-08,Zorrosdeldesiertocobreloa@gmail.com,Titular,1,119907888119907888
1,Construmart,14099541-K,14099541-K,F,TITULAR,Zulema Suhellen,Diaz Baez,1981-10-11,zsdbaez@gmail.com,Titular,1,14099541K14099541K
2,Construmart,10016990-8,10016990-8,F,TITULAR,Fideliza Albertina,Oyarzo Antilef,1964-12-27,zitaantilef@gmail.com,Titular,1,100169908100169908
3,Construmart,11151422-4,11151422-4,F,TITULAR,Zunilda Del Rosario,Escobar Cabezas,1967-08-14,zescobar@construmart.cl,Titular,1,111514224111514224
4,Construmart,18761523-2,18761523-2,M,TITULAR,Cristian Ignacio,Plaza Tapia,1995-02-23,ZAPLA.DH@GMAIL.COM,Titular,1,187615232187615232


In [182]:
Pacientes_Archivo.shape

(3368, 12)

In [183]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",        
        "Id Contacto",
    ]
]

In [184]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\3487171931.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


(3368, 13)

In [185]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\980738941.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(3368, 14)

In [186]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()



C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\3035579401.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\3035579401.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [187]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo

#Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
#Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

,Empresa,rut2,rut1,sexo,ParentesCONYUGE,nombres,apellidos,nacimiento,email,Relacion,Activo,Llave,rut,rutT
0,Construmart,11990788-8,11990788-8,M,TITULAR,Ignacio Enrique,Arancibia Silva,1972-03-08,Zorrosdeldesiertocobreloa@gmail.com,Titular,1,119907888119907888,119907888,119907888
1,Construmart,14099541-K,14099541-K,F,TITULAR,Zulema Suhellen,Diaz Baez,1981-10-11,zsdbaez@gmail.com,Titular,1,14099541K14099541K,14099541K,14099541K
2,Construmart,10016990-8,10016990-8,F,TITULAR,Fideliza Albertina,Oyarzo Antilef,1964-12-27,zitaantilef@gmail.com,Titular,1,100169908100169908,100169908,100169908
3,Construmart,11151422-4,11151422-4,F,TITULAR,Zunilda Del Rosario,Escobar Cabezas,1967-08-14,zescobar@construmart.cl,Titular,1,111514224111514224,111514224,111514224
4,Construmart,18761523-2,18761523-2,M,TITULAR,Cristian Ignacio,Plaza Tapia,1995-02-23,ZAPLA.DH@GMAIL.COM,Titular,1,187615232187615232,187615232,187615232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3363,Construmart,16631809-2,20962597-0,F,Hijo I.P.,Lisbeth Andr,Martinez Martinez,NaT,GHISLENS.1982@GMAIL.COM,Carga,1,166318092209625970,209625970,166318092
3364,Construmart,13498946-7,22357415-7,M,Hijo,Martin Felipe,Menendez Lopez,NaT,DMENENDEZM@GMAIL.COM,Carga,1,134989467223574157,223574157,134989467
3365,Construmart,17860480-5,24346142-1,M,Hijo,Maximo Vicente,Ortega Nadeau,NaT,IVANNIENADEAU@GMAIL.COM,Carga,1,178604805243461421,243461421,178604805
3366,Construmart,10218788-1,10119898-7,F,Conyuge,Ana Maria,Moscoso Carmona,NaT,RICARDO.PARDOD07@GMAIL.COM,Carga,1,102187881101198987,101198987,102187881


In [188]:
#Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo22.xlsx", index=False) 

In [189]:
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") ]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rutT"] != "NONE") ]
#| (Pacientes_Archivo["rutT"] != "NONE")

PAcientes Programas

In [190]:



query_pacientePrograma = "SELECT Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id from Paciente_Programas__c where padre__c =  '" + campaña +"'"
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()

100%|██████████| 1/1 [00:05<00:00,  5.82s/it]


,Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id
0,0018c00002ghJVnAAM,0018c00002ghJeUAAU,162574019,a1Y8c00000DxgpeEAB
1,0018c00002ghJXsAAM,0018c00002ghJeUAAU,111935378,a1Y8c00000DxgpjEAB
2,0018c00002nl5K3AAI,0018c00002ghJeUAAU,192445027,a1Y8c00000EHn2GEAT
3,001RN0000027o3lYAA,0018c00002ghJeUAAU,19445049,a1YRN000000BKeX2AW
4,001RN0000027o3mYAA,0018c00002ghJeUAAU,157414577,a1YRN000000BKeY2AW


In [191]:
pacientePrograma = pacientePrograma.rename(
    columns={
        "Id": "Id Paciente Programas",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Programas_por_Empresas__r.Empresa__c": "Id Account Empleador",
        "Paciente__c": "Id Account Paciente",
    }
)
pacientePrograma.head()

,Id Account Paciente,Id Account Empleador,Rut,Id Paciente Programas
0,0018c00002ghJVnAAM,0018c00002ghJeUAAU,162574019,a1Y8c00000DxgpeEAB
1,0018c00002ghJXsAAM,0018c00002ghJeUAAU,111935378,a1Y8c00000DxgpjEAB
2,0018c00002nl5K3AAI,0018c00002ghJeUAAU,192445027,a1Y8c00000EHn2GEAT
3,001RN0000027o3lYAA,0018c00002ghJeUAAU,19445049,a1YRN000000BKeX2AW
4,001RN0000027o3mYAA,0018c00002ghJeUAAU,157414577,a1YRN000000BKeY2AW


In [192]:
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("-", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("–", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.upper()
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.strip()
pacientePrograma.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\3172767859.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")


(4135, 4)

In [193]:

PacientesProgramas_PacientesSF = pd.merge(
    left=pacientePrograma,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)

PacientesProgramas_PacientesSF.head()

,Id Account Paciente,Id Account Empleador,Rut_x,Id Paciente Programas,Rut_y,Id Contacto
0,0018c00002ghJVnAAM,0018c00002ghJeUAAU,162574019,a1Y8c00000DxgpeEAB,162574019,0038c00003BXaaXAAT
1,0018c00002ghJXsAAM,0018c00002ghJeUAAU,111935378,a1Y8c00000DxgpjEAB,111935378,0038c00003BXac4AAD
2,0018c00002nl5K3AAI,0018c00002ghJeUAAU,192445027,a1Y8c00000EHn2GEAT,192445027,0038c00003JnT0pAAF
3,001RN0000027o3lYAA,0018c00002ghJeUAAU,19445049,a1YRN000000BKeX2AW,19445049,003RN000004BrjaYAC
4,001RN0000027o3mYAA,0018c00002ghJeUAAU,157414577,a1YRN000000BKeY2AW,157414577,003RN000004BrjbYAC


account contact relation

In [194]:

query_accountContact = "select Id,ContactId, AccountId, RutContact__c ,Contact.AccountId from AccountContactRelation where Account.Campa_a__c = '" + campaña +"'"
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()

100%|██████████| 1/1 [00:06<00:00,  6.38s/it]


,Id,ContactId,AccountId,RutContact__c,Contact.AccountId
0,07k8c00000ZA5UVAA1,0038c00003BXaaXAAT,0018c00002ghJeUAAU,162574019,0018c00002ghJVnAAM
1,07k8c00000ZA5UWAA1,0038c00003BXac4AAD,0018c00002ghJeUAAU,111935378,0018c00002ghJXsAAM
2,07k8c00000aFERNAA4,0038c00003JnT0pAAF,0018c00002ghJeUAAU,192445027,0018c00002nl5K3AAI
3,07kRN000001Cb6fYAC,003RN000004BrjaYAC,0018c00002ghJeUAAU,19445049,001RN0000027o3lYAA
4,07kRN000001Cb6gYAC,003RN000004BrjbYAC,0018c00002ghJeUAAU,157414577,001RN0000027o3mYAA


In [195]:
accountContactRelation = accountContactRelation.rename(
    columns={
        "Id": "Id AccountContactRelation",
        "ContactId": "Id Contact Paciente",
        "AccountId": "Id Account Empresa",
        "RutContact__c": "Rut",
        "Contact.AccountId":"Id Account Paciente"
    }
)
accountContactRelation.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut,Id Account Paciente
0,07k8c00000ZA5UVAA1,0038c00003BXaaXAAT,0018c00002ghJeUAAU,162574019,0018c00002ghJVnAAM
1,07k8c00000ZA5UWAA1,0038c00003BXac4AAD,0018c00002ghJeUAAU,111935378,0018c00002ghJXsAAM
2,07k8c00000aFERNAA4,0038c00003JnT0pAAF,0018c00002ghJeUAAU,192445027,0018c00002nl5K3AAI
3,07kRN000001Cb6fYAC,003RN000004BrjaYAC,0018c00002ghJeUAAU,19445049,001RN0000027o3lYAA
4,07kRN000001Cb6gYAC,003RN000004BrjbYAC,0018c00002ghJeUAAU,157414577,001RN0000027o3mYAA


In [196]:
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("-", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("–", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.upper()
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.strip()
accountContactRelation.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\2268980097.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")


(4141, 5)

In [197]:
AccountContact_PacientesSF = pd.merge(
    left=accountContactRelation,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
AccountContact_PacientesSF.head()


,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut_x,Id Account Paciente,Rut_y,Id Contacto
0,07k8c00000ZA5UVAA1,0038c00003BXaaXAAT,0018c00002ghJeUAAU,162574019,0018c00002ghJVnAAM,162574019,0038c00003BXaaXAAT
1,07k8c00000ZA5UWAA1,0038c00003BXac4AAD,0018c00002ghJeUAAU,111935378,0018c00002ghJXsAAM,111935378,0038c00003BXac4AAD
2,07k8c00000aFERNAA4,0038c00003JnT0pAAF,0018c00002ghJeUAAU,192445027,0018c00002nl5K3AAI,192445027,0038c00003JnT0pAAF
3,07kRN000001Cb6fYAC,003RN000004BrjaYAC,0018c00002ghJeUAAU,19445049,001RN0000027o3lYAA,19445049,003RN000004BrjaYAC
4,07kRN000001Cb6gYAC,003RN000004BrjbYAC,0018c00002ghJeUAAU,157414577,001RN0000027o3mYAA,157414577,003RN000004BrjbYAC


empresa contacto convenio

In [198]:
query_empresacontactoconvenio = "select Id,Paciente__r.IdentificationId__pc,  Empresa__c, Paciente__c,N_Documento_Titular__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '" + campaña +"'"
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()

100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


,Id,Paciente__r.IdentificationId__pc,Empresa__c,Paciente__c,N_Documento_Titular__c
0,a40RN000000vtW1YAI,160419121,a3yRN0000006rlZYAQ,0018c00002giNDwAAM,160419121


In [199]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Empresa__c": "Id Convenio Empresa",
        "Paciente__c": "Id Account Paciente",
        "N_Documento_Titular__c":"Rut Titular"
    }
)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Rut,Id Convenio Empresa,Id Account Paciente,Rut Titular
0,a40RN000000vtW1YAI,160419121,a3yRN0000006rlZYAQ,0018c00002giNDwAAM,160419121


In [200]:
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("-", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("–", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.upper()
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.strip()
empresacontactoconvenio.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\2534521553.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")


(1, 5)

In [201]:
empresacontactoconvenio_PacientesSF = pd.merge(
    left=empresacontactoconvenio,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Id Convenio Empresa,Id Account Paciente,Rut Titular,Rut_y,Id Contacto
0,a40RN000000vtW1YAI,160419121,a3yRN0000006rlZYAQ,0018c00002giNDwAAM,160419121,160419121,0038c00003C43B8AAJ


In [202]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

' if(campaña!="MetLife"):\n    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) '

Rescatamos todos los ruts Id con su empresa desde la campaña

In [203]:
Pacientes_Archivo.head()

,Empresa,rut2,rut1,sexo,ParentesCONYUGE,nombres,apellidos,nacimiento,email,Relacion,Activo,Llave,rut,rutT
0,Construmart,11990788-8,11990788-8,M,TITULAR,Ignacio Enrique,Arancibia Silva,1972-03-08,Zorrosdeldesiertocobreloa@gmail.com,Titular,1,119907888119907888,119907888,119907888
1,Construmart,14099541-K,14099541-K,F,TITULAR,Zulema Suhellen,Diaz Baez,1981-10-11,zsdbaez@gmail.com,Titular,1,14099541K14099541K,14099541K,14099541K
2,Construmart,10016990-8,10016990-8,F,TITULAR,Fideliza Albertina,Oyarzo Antilef,1964-12-27,zitaantilef@gmail.com,Titular,1,100169908100169908,100169908,100169908
3,Construmart,11151422-4,11151422-4,F,TITULAR,Zunilda Del Rosario,Escobar Cabezas,1967-08-14,zescobar@construmart.cl,Titular,1,111514224111514224,111514224,111514224
4,Construmart,18761523-2,18761523-2,M,TITULAR,Cristian Ignacio,Plaza Tapia,1995-02-23,ZAPLA.DH@GMAIL.COM,Titular,1,187615232187615232,187615232,187615232


Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

Pacientes_Archivo.drop_duplicates(
    [
        "rut",
        "rutT",
        "LLaveEmpresa",
        "LLAveRut",
    ],
    keep="last",
)
Pacientes_Archivo.shape

In [204]:
print(campaña)

Construmart


In [235]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [236]:
#Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo.xlsx", index=False)

In [237]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
#Pacientes_con_id.to_excel("F:/Empresas SF/reportes/Pacientes_con_id.xlsx", index=False)
Pacientes_con_id.shape

(3367, 25)

In [208]:
Pacientes_con_id.head()

,Empresa,rut2,rut1,sexo,ParentesCONYUGE,nombres,apellidos,nacimiento,email,Relacion,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
0,Construmart,11990788-8,11990788-8,M,TITULAR,Ignacio Enrique,Arancibia Silva,1972-03-08,Zorrosdeldesiertocobreloa@gmail.com,Titular,...,0018c00002giNCtAAM,119907888,Ignacio Enrique Arancibia Silva,RUT,None,None,None,Hombre,None,0038c00003C43A5AAJ
1,Construmart,14099541-K,14099541-K,F,TITULAR,Zulema Suhellen,Diaz Baez,1981-10-11,zsdbaez@gmail.com,Titular,...,0018c00002giNCuAAM,14099541K,Zulema Suhellen Diaz Baez,RUT,None,None,None,Mujer,None,0038c00003C43A6AAJ
2,Construmart,10016990-8,10016990-8,F,TITULAR,Fideliza Albertina,Oyarzo Antilef,1964-12-27,zitaantilef@gmail.com,Titular,...,0018c00002giNCvAAM,100169908,Fideliza Albertina Oyarzo Antilef,RUT,None,None,None,Mujer,None,0038c00003C43A7AAJ
3,Construmart,11151422-4,11151422-4,F,TITULAR,Zunilda Del Rosario,Escobar Cabezas,1967-08-14,zescobar@construmart.cl,Titular,...,0018c00002giNCwAAM,111514224,Zunilda Del Rosario Escobar Cabezas,RUT,None,None,None,Mujer,None,0038c00003C43A8AAJ
4,Construmart,18761523-2,18761523-2,M,TITULAR,Cristian Ignacio,Plaza Tapia,1995-02-23,ZAPLA.DH@GMAIL.COM,Titular,...,0018c00002giNCyAAM,187615232,Cristian Ignacio Plaza Tapia,RUT,None,None,None,Hombre,None,0038c00003C43AAAAZ


In [238]:
Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

In [209]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):        
        return email        
    else:        
        return ""
    



In [239]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
#Subir_pacientes["Email_Seteado__c"] = ""
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )


if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = pd.to_datetime(Pacientes_con_id["nacimiento"],format="%Y-%m-%d")

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"
    
Subir_pacientes_Que_no_Estan =Subir_pacientes
Subir_pacientes_Estan = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]
Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]



# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
3037,False,Gaspar Ismae,Castro Briceño,280488046,,False,2023-01-21,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3038,False,Mateo Salvador Benito,Castro Briceño,280488321,,False,2023-01-22,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3039,False,Viviana Lissette Del Carmen,Castro Moya,167141986,,False,1987-05-23,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [248]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"
    
def traerContact(ide):
    if(len(ide)>=12):
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"
    

def agregarPaciente(listado):
    vListado = listado.split(",")
    derty = dict(
        sf.Account.create(
            # derty = sf.Account.create(
            {
                "FirstName": vListado[0],
                "LastName": vListado[1],
                "IdentificationId__pc": vListado[2],
                "RecordTypeId": vListado[3],
                "BillingCountry": vListado[4],
                "Identification_Type__c": vListado[5],
                "Pa_s_del_Paciente__c": vListado[6],
                "Cuenta_validada__c": True,
                "PersonEmail": vListado[8],
                "PersonBirthdate": pd.to_datetime(vListado[9]).isoformat() + 'Z',
                "HealthCloudGA__Gender__pc": vListado[10],
                "Phone": vListado[11],
                "BillingCity": vListado[12],
                "BillingState": vListado[13],
                "BillingStreet": vListado[14],
                "Email_Seteado__c": bool(vListado[15]),
            }
        )
    )
    return derty["id"]

In [227]:
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
3037,False,Gaspar Ismae,Castro Briceño,280488046,,False,2023-01-21,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3038,False,Mateo Salvador Benito,Castro Briceño,280488321,,False,2023-01-22,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3039,False,Viviana Lissette Del Carmen,Castro Moya,167141986,,False,1987-05-23,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [228]:

Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
        
    ],
    keep="last",
)

Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)

Subir_pacientes.shape

001RN000005TFiLYAW 280488046
001RN000005TAM8YAO 280488321
001RN000005TFjxYAG 167141986


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\1714459819.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(


(3, 14)

In [229]:
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() == 15]

In [230]:
#Subir_pacientes["dato_account"].head()

In [231]:
#Subir_pacientes.to_excel("F:/Empresas SF/reportes/Todesffff.xlsx", index=False)


In [232]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes["FirstName"].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["IdentificationId__pc"].astype(str)
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["RecordTypeId"].astype(str)
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingCountry"].astype(str)
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Identification_Type__c"].astype(str)
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Cuenta_validada__c"].astype(str)

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonEmail"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," 
    
if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].astype(str) + " 00:00:00"
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","      
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","   
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingCity"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingState"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingStreet"].astype(str) 
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," 
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Email_Seteado__c"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  

#Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """



' Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(\n    lambda x: agregarPaciente(x)\n) '

In [243]:
Pacientes_con_id_no.head()

,Empresa,rut2,rut1,sexo,ParentesCONYUGE,nombres,apellidos,nacimiento,email,Relacion,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
3037,Construmart,17278862-9,28048804-6,Hombre,HIJO,Gaspar Ismae,Castro Briceño,2023-01-21,NaN,Carga,...,001RN000005TFiLYAW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3038,Construmart,17278862-9,28048832-1,Hombre,HIJO,Mateo Salvador Benito,Castro Briceño,2023-01-22,NaN,Carga,...,001RN000005TAM8YAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3039,Construmart,17278862-9,16714198-6,Mujer,Señora,Viviana Lissette Del Carmen,Castro Moya,1987-05-23,NaN,Carga,...,001RN000005TFjxYAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [250]:
Pacientes_con_id_no = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == False]
Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
    lambda y: traerId(y)
)
Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
    lambda y: traerContact(y)
)
Pacientes_con_id_no.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_no.xlsx", index=False
)

Pacientes_con_id_si = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == True]
Pacientes_con_id_si.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_si.xlsx", index=False
)

Pacientes_con_id_final = pd.concat(
    [
        Pacientes_con_id_si,
        Pacientes_con_id_no,
    ],
    axis=0,
)

Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final.xlsx", index=False
)

if campaña == "Southbridge Plan Mujer":
    VpacientePrograma = "a1WRN0000007YwD2AU"
    VempresaaAccount = "001RN000003rwVaYAI"
    VempresaConvenio = "a3yRN0000006sVTYAY"
if campaña == "Southbridge":
    VpacientePrograma = "a1W8c0000083pM9EAI"
    VempresaaAccount = "0018c00002f3Ev2AAE"
    VempresaConvenio = "a3yRN0000006sqYYAQ"
if campaña == "Construmart":
    VpacientePrograma = "a1W8c0000083q9TEAQ"
    VempresaaAccount = "0018c00002ghJgBAAU"
    VempresaConvenio = "a3yRN0000006rlZYAQ"

else:
    VpacientePrograma = "Buscarv"
    VempresaaAccount = "Buscarv"
    VempresaConvenio = "Buscarv"


Pacientes_con_id_final["esonoes"] = Pacientes_con_id_final["rut"].isin(
    PacientesProgramas_PacientesSF["Rut_y"]
)
Subir_programas_empresa = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes"] == False
]
Subir_programas_empresa["Paciente__c"] = Subir_programas_empresa["Id Account Paciente"]
Subir_programas_empresa["Programas_por_Empresas__c"] = VpacientePrograma
Subir_programas_empresa = Subir_programas_empresa[
    [
        # "empresa",
        # "poliza",
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Subir_programas_empresa.to_excel(
    "F:/Empresas SF/reportes/programas_empresa.xlsx", index=False
)


Pacientes_con_id_final["esonoes1"] = Pacientes_con_id_final["rut"].isin(
    AccountContact_PacientesSF["Rut_y"]
)
Subir_AccountContact = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes1"] == False
]
Subir_AccountContact["ContactId"] = Subir_AccountContact["Id Contacto"]
Subir_AccountContact["AccountId"] = VempresaaAccount

Subir_AccountContact = Subir_AccountContact[
    [
        # "empresa",
        # "poliza",
        "ContactId",
        "AccountId",
    ]
]
Subir_AccountContact.to_excel(
    "F:/Empresas SF/reportes/account_Contact.xlsx", index=False
)


Pacientes_con_id_final["esonoes2"] = Pacientes_con_id_final["rut"].isin(
    empresacontactoconvenio_PacientesSF["Rut_y"]
)
Subir_empresacontactoconvenio = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes2"] == False
]
Subir_empresacontactoconvenio["Paciente__c"] = Subir_empresacontactoconvenio[
    "Id Account Paciente"
]
Subir_empresacontactoconvenio["N_Documento_Titular__c"] = Subir_empresacontactoconvenio[
    "rutT"
]
Subir_empresacontactoconvenio["Empresa__c"] = VempresaConvenio

Subir_empresacontactoconvenio = Subir_empresacontactoconvenio[
    [
        # "empresa",
        # "poliza",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
    ]
]
Subir_empresacontactoconvenio.to_excel(
    "F:/Empresas SF/reportes/empresacontactoconvenio.xlsx", index=False
)

001RN000005TFiLYAW 280488046
001RN000005TAM8YAO 280488321
001RN000005TFjxYAG 167141986


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\1742322872.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\1742322872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9088\1742322872.py:54: SettingWithCopyWarning: 
A value is trying 

In [246]:
Pacientes_con_id_no.head()

,Empresa,rut2,rut1,sexo,ParentesCONYUGE,nombres,apellidos,nacimiento,email,Relacion,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
3037,Construmart,17278862-9,28048804-6,Hombre,HIJO,Gaspar Ismae,Castro Briceño,2023-01-21,NaN,Carga,...,001RN000005TFiLYAW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3038,Construmart,17278862-9,28048832-1,Hombre,HIJO,Mateo Salvador Benito,Castro Briceño,2023-01-22,NaN,Carga,...,001RN000005TAM8YAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3039,Construmart,17278862-9,16714198-6,Mujer,Señora,Viviana Lissette Del Carmen,Castro Moya,1987-05-23,NaN,Carga,...,001RN000005TFjxYAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
contactd = sf.Account.get('0018c00002lUyo0AAC')
contactd["PersonContactId"]


In [ ]:
#contactd["PersonContactId"]
print (datetime.now())

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/todes.xlsx", index=False)

query2 = "select P_liza__c, Nombre_de_Empresa__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)

derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId__pc": "derty", "Email": "example@example.com"})

Subir_pacientes.head()

Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account__l.xlsx", index=False)

In [ ]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

In [ ]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()